In [3]:
from pydantic import BaseModel, ConfigDict, ValidationError
from enum import Enum


In [4]:
class Color(Enum):
    red = "Red"
    green = "Green"
    blue = "Blue"


In [5]:
Color.red


<Color.red: 'Red'>

In [6]:
Color.red.value


'Red'

In [7]:
class Model(BaseModel):
    color: Color


In [9]:
# Serialize
Model(color=Color.red)


Model(color=<Color.red: 'Red'>)

In [10]:
# Deserialize
data = '''{ "color": "Red" }'''
Model.model_validate_json(json_data=data)


Model(color=<Color.red: 'Red'>)

#### Model dump 를 하면

In [11]:
m = Model(color=Color.red)


In [12]:
m.model_dump()  # value 의 Enum 이 보존됨 --> 그런데 {'color': 'Red'} 로 serialize 되길 바란다면?


{'color': <Color.red: 'Red'>}

In [14]:
m.model_dump_json()  # value 의 String 값으로 serialize


'{"color":"Red"}'

#### model_dump() 때도 Enum 이 아닌 model_dump_json() 시의 String 으로 serialize 되길 원한다면

In [15]:
class Model2(BaseModel):
    model_config = ConfigDict(use_enum_values=True)  # 이걸 설정하면

    color: Color


In [17]:
m2 = Model2(color=Color.red)
m2.model_dump()


{'color': 'Red'}

#### 그런데 use_enum_values=True 가 설정되었더라도, default 로 설정된 값은 제대로 이 설정이 적용되지 않음 ㅠ


In [19]:
class Model3(BaseModel):
    model_config = ConfigDict(use_enum_values=True)

    color: Color = Color.red


In [23]:
m3 = Model3()
m3.model_dump()  # 'Red' 가 아닌 <Color:red: 'Red'> 로 표현됨 !!!

# 이건 pydantic 이 default value 를 체크하지 않기 때문


{'color': <Color.red: 'Red'>}

#### 위의 문제를 해결할려면


In [25]:
## 방법1
class Model3(BaseModel):
    model_config = ConfigDict(use_enum_values=True, validate_default=True)

    color: Color = Color.red

Model3()


Model3(color='Red')

In [27]:
## 방법2

class Model3(BaseModel):
    model_config = ConfigDict(use_enum_values=True)

    color: Color = Color.red.value

Model3()


Model3(color='Red')